# IMPORTS

In [1]:
from lightgbm import LGBMRanker, LGBMRegressor
from scipy.special import softmax

import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Load datasets

In [2]:
visits = pd.read_parquet('train_visits.parquet.gzip')
ports = pd.read_parquet('ports_prep.parquet.gzip')
vessels = pd.read_parquet('vessels_prep.parquet.gzip')

ports = ports.set_index('port_index')

# Define the dataframe
- Select the instances where NO target is present (i.e. the data we want to predict)
- Sort values for the split and take the first few so that we only predict the most recent ones (i.e. the most interesting ones)
- Drop irrelevant columns
- Add an identifier (in this case just the mmsi, since there is only one visit without a target for each mmsi)

In [4]:
visits = visits[(visits['target_port_index'].isnull()) &
      (visits['prev2_port_index'].notnull())].sort_values('exit_datetime', ascending=False).head(25000)


df = df.sample(2500)

In [5]:
df = df.drop(['entry_datetime', 'target_entry_datetime', 
                      'previous_exit_datetime',
                      'previous_portname', 'previous_entry_datetime', 
                      'prev2_exit_datetime'], 1)

df['identifier'] = df['mmsi']

# Create dataset for ranking
- Get pivots; creates pivot tables with new features
- Create dataset; creates a dataset for ranking purposes. Each instance is expanded to ~1800 instances each with a different potential target port.
- add features; adds new features (also from the pivots) to the dataset

In [6]:
def get_pivots(port_only=False):
    #add iso3 codes to hist_visits
    ports=pd.read_parquet('ports_prep.parquet.gzip').set_index('port_index')
    hist_visits = pd.read_parquet('hist_visits.parquet.gzip')
    hist_visits = pd.merge(hist_visits, ports[['iso3']].reset_index(), how='left', on='port_index')
    hist_visits = pd.merge(hist_visits, ports[['iso3']].rename(columns={'iso3':'target_iso3'}), 
                               how='left', left_on='target_port_index', right_index=True)
    
    hist_visits['hist_connections'] = 1    
    
    #Connections between ports
    port_pt = pd.pivot_table(hist_visits, index=['port_index', 'target_port_index'], 
                        values='hist_connections', aggfunc='count').fillna(0).reset_index()
    port_pt.rename(columns={'hist_connections': 'port_hist_connections'}, inplace=True)
    port_pt = port_pt.set_index(['port_index', 'target_port_index'])
    
    if port_only:
        return port_pt
    
    #travel time/stay duration between ports
    port_pt2 = pd.pivot_table(hist_visits, index=['port_index', 'target_port_index'], 
                        values=['target_travel_duration', 'target_stay_duration'], 
                         aggfunc=['mean', 'std', 'max', 'min']).reset_index()
    port_pt2 = port_pt2.set_index(['port_index', 'target_port_index'])
    port_pt2.columns = ['ports_' + '_'.join((str(j), str(k))) for j, k in port_pt2.columns]  
    
    
    #connections between countries
    iso_pt = pd.pivot_table(hist_visits, index=['iso3', 'target_iso3'], 
                        values='hist_connections', aggfunc='count').fillna(0).reset_index()
    iso_pt.rename(columns={'hist_connections': 'iso_hist_connections'}, inplace=True)
    iso_pt = iso_pt.set_index(['iso3', 'target_iso3'])

    #travel time/stay duration between countries
    iso_pt2 = pd.pivot_table(hist_visits, index=['iso3', 'target_iso3'], 
                        values=['target_travel_duration', 'target_stay_duration'], 
                         aggfunc=['mean', 'std', 'max', 'min']).reset_index()
    iso_pt2 = iso_pt2.set_index(['iso3', 'target_iso3'])
    iso_pt2.columns = ['iso3_' + '_'.join((str(j), str(k))) for j, k in iso_pt2.columns]    
    

    #merge datasets
    port_pt = pd.merge(port_pt, port_pt2, left_index=True, right_index=True)
    iso_pt = pd.merge(iso_pt, iso_pt2, left_index=True, right_index=True)

    
    #previous port --> target port connections
    prev_pt = pd.pivot_table(hist_visits, index=['previous_port_index', 'target_port_index'], 
                        values='hist_connections', aggfunc='count').fillna(0).reset_index()
    prev_pt.rename(columns={'hist_connections': 'prev_hist_connections'}, inplace=True)
    prev_pt = prev_pt.set_index(['previous_port_index', 'target_port_index'])
    
    #prev2 port --> target port connections
    prev2_pt = pd.pivot_table(hist_visits, index=['prev2_port_index', 'target_port_index'], 
                            values='hist_connections', aggfunc='count').fillna(0).reset_index()
    prev2_pt.rename(columns={'hist_connections': 'prev2_hist_connections'}, inplace=True)
    prev2_pt = prev2_pt.set_index(['prev2_port_index', 'target_port_index'])
    
    return port_pt, iso_pt, prev_pt, prev2_pt

In [7]:
def create_dataset(data):
    samples = len(data)
    data = data.iloc[np.repeat(np.arange(len(data)), len(ports))]
    data['target_port_index'] = np.tile(ports.index.astype('int').values, samples)
    
    #data = data[data['port_index'] != data['target_port_index']]
    
    return data

In [8]:
def add_features(dataset):
    port_pt, iso_pt, prev_pt, prev2_pt = get_pivots()
    ports = pd.read_parquet('ports_prep.parquet.gzip').set_index('port_index')
    distances = pd.read_excel('CERDI.xlsx').set_index(['iso1', 'iso2'])[['seadistance']]
    og_port_columns = ports.columns
    
    #add target port features
    ports.columns = ['target_' + column for column in og_port_columns]
    dataset = pd.merge(dataset, ports, how='left', left_on='target_port_index', right_index=True)
    ports.columns = og_port_columns
    
    #add port_pt, iso_pt features
    dataset = pd.merge(dataset, port_pt, how='left', left_on=['port_index', 'target_port_index'], right_index=True)
    dataset = pd.merge(dataset, iso_pt, how='left', left_on=['iso3', 'target_iso3'], right_index=True)
    dataset['port_hist_connections'] = dataset['port_hist_connections'].fillna(0)
    dataset['iso_hist_connections'] = dataset['iso_hist_connections'].fillna(0)
    
    #add fav port connections
    port_pt = port_pt.reset_index().groupby('port_index').head(1)
    dataset = pd.merge(dataset, port_pt[['port_index', 'target_port_index', 
                       'port_hist_connections']].rename(columns={'target_port_index': 'port_fav_port',
                       'port_hist_connections': 'port_fav_port_count'}), 
                       how='left', on='port_index')
    dataset['port_fav_port_count'] = dataset['port_fav_port_count'].fillna(0)
    dataset['port_fav_port'] = dataset['port_fav_port'].astype('object').fillna('Unknown').astype('category')

    ports.columns = ['port_fav_' + column for column in og_port_columns]
    dataset = pd.merge(dataset, ports[['port_fav_port_lat', 'port_fav_port_long']], 
                       how='left', left_on='target_port_index', right_index=True)
    ports.columns = og_port_columns
    
    #add previous and prev2 pt features
    dataset = pd.merge(dataset, prev_pt, how='left', left_on=['previous_port_index', 'target_port_index'], right_index=True)
    dataset = pd.merge(dataset, prev2_pt, how='left', left_on=['prev2_port_index', 'target_port_index'], right_index=True)
    dataset['prev_hist_connections'] = dataset['prev_hist_connections'].fillna(0)
    dataset['prev2_hist_connections'] = dataset['prev2_hist_connections'].fillna(0)    
    
    #add seadistance curr -- target
    dataset = pd.merge(dataset, distances[['seadistance']], how='left',
                       left_on=['iso3', 'target_iso3'], right_index=True)
    dataset['seadistance'] = dataset['seadistance'].fillna(0)

    #add seadistance prev -- target
    dataset = pd.merge(dataset, distances[['seadistance']].rename(columns={'seadistance': 'prev_tar_seadistance'}), 
                       how='left', left_on=['previous_iso3', 'target_iso3'], right_index=True)
    dataset['prev_tar_seadistance'] = dataset['prev_tar_seadistance'].fillna(0)

    #add seadistance prev2 -- target
    dataset = pd.merge(dataset, distances[['seadistance']].rename(columns={'seadistance': 'prev2_tar_seadistance'}), 
                       how='left', left_on=['prev2_iso3', 'target_iso3'], right_index=True)
    dataset['prev2_tar_seadistance'] = dataset['prev2_tar_seadistance'].fillna(0)
    
    #add euclidean distances
    dataset['eucl_cur_tar'] = np.sqrt((dataset['port_lat'] - dataset['target_port_lat'])**2 + 
                                 (dataset['port_long'] - dataset['target_port_long'])**2)
    dataset['eucl_cur_prev'] = np.sqrt((dataset['port_lat'] - dataset['previous_port_lat'])**2 + 
                                 (dataset['port_long'] - dataset['previous_port_long'])**2)
    dataset['eucl_tar_prev'] = np.sqrt((dataset['target_port_lat'] - dataset['previous_port_lat'])**2 + 
                                 (dataset['target_port_long'] - dataset['previous_port_long'])**2)
    dataset['eucl_ves_tar'] = np.sqrt((dataset['mean_port_lat'] - dataset['target_port_lat'])**2 + 
                                 (dataset['mean_port_long'] - dataset['target_port_long'])**2)
    dataset['eucl_ves_cur'] = np.sqrt((dataset['mean_port_lat'] - dataset['port_lat'])**2 + 
                                 (dataset['mean_port_long'] - dataset['port_long'])**2)
    dataset['eucl_ves_prev'] = np.sqrt((dataset['mean_port_lat'] - dataset['previous_port_lat'])**2 + 
                                 (dataset['mean_port_long'] - dataset['previous_port_long'])**2)
    
    dataset['eucl_cur_prev2'] = np.sqrt((dataset['port_lat'] - dataset['prev2_port_lat'])**2 + 
                                 (dataset['port_long'] - dataset['prev2_port_long'])**2)
    dataset['eucl_tar_prev2'] = np.sqrt((dataset['target_port_lat'] - dataset['prev2_port_lat'])**2 + 
                                 (dataset['target_port_long'] - dataset['prev2_port_long'])**2)
    dataset['eucl_prev_prev2'] = np.sqrt((dataset['previous_port_lat'] - dataset['prev2_port_lat'])**2 + 
                                 (dataset['previous_port_long'] - dataset['prev2_port_long'])**2)
    dataset['eucl_ves_prev2'] = np.sqrt((dataset['mean_port_lat'] - dataset['prev2_port_lat'])**2 + 
                                 (dataset['mean_port_long'] - dataset['prev2_port_long'])**2)    

    dataset['eucl_ves_fav_cur'] = np.sqrt((dataset['vessel_fav_port_lat'] - dataset['port_lat'])**2 + 
                                 (dataset['vessel_fav_port_long'] - dataset['port_long'])**2)
    dataset['eucl_ves_fav_tar'] = np.sqrt((dataset['vessel_fav_port_lat'] - dataset['target_port_lat'])**2 + 
                                 (dataset['vessel_fav_port_long'] - dataset['target_port_long'])**2)
    dataset['eucl_ves_fav_prev'] = np.sqrt((dataset['vessel_fav_port_lat'] - dataset['previous_port_lat'])**2 + 
                                 (dataset['vessel_fav_port_long'] - dataset['previous_port_long'])**2)
    dataset['eucl_ves_fav_prev2'] = np.sqrt((dataset['vessel_fav_port_lat'] - dataset['prev2_port_lat'])**2 + 
                                 (dataset['vessel_fav_port_long'] - dataset['prev2_port_long'])**2) 
    dataset['eucl_ves_fav_ves'] = np.sqrt((dataset['vessel_fav_port_lat'] - dataset['mean_port_lat'])**2 + 
                                 (dataset['vessel_fav_port_long'] - dataset['mean_port_long'])**2)
    
    dataset['eucl_port_fav_tar'] = np.sqrt((dataset['port_fav_port_lat'] - dataset['target_port_lat'])**2 + 
                                 (dataset['port_fav_port_long'] - dataset['target_port_long'])**2)
    dataset['eucl_port_fav_prev'] = np.sqrt((dataset['port_fav_port_lat'] - dataset['previous_port_lat'])**2 + 
                                 (dataset['port_fav_port_long'] - dataset['previous_port_long'])**2)
    dataset['eucl_port_fav_ves'] = np.sqrt((dataset['port_fav_port_lat'] - dataset['mean_port_lat'])**2 + 
                                 (dataset['port_fav_port_long'] - dataset['mean_port_long'])**2)    
    dataset['eucl_port_fav_ves_fav'] = np.sqrt((dataset['port_fav_port_lat'] - dataset['vessel_fav_port_lat'])**2 + 
                                 (dataset['port_fav_port_long'] - dataset['vessel_fav_port_long'])**2)   
    
    #add rankings
    dataset['rank_port_connections'] = dataset.groupby('identifier')['port_hist_connections'].rank(axis=0, 
                                                                                        method='min', ascending=False)
    dataset['rank_iso_connections'] = dataset.groupby('identifier')['iso_hist_connections'].rank(axis=0, 
                                                                                        method='min', ascending=False)
    dataset['rank_prev_connections'] = dataset.groupby('identifier')['prev_hist_connections'].rank(axis=0, 
                                                                                        method='min', ascending=False)
    dataset['rank_prev2_connections'] = dataset.groupby('identifier')['prev2_hist_connections'].rank(axis=0, 
                                                                                        method='min', ascending=False)
    dataset['rank_seadistance'] = dataset.groupby('identifier')['seadistance'].rank(axis=0, method='min')
    dataset['rank_prev_tar_seadistance'] = dataset.groupby('identifier')['prev_tar_seadistance'].rank(axis=0, method='min')
    dataset['rank_eucl_cur_tar'] = dataset.groupby('identifier')['eucl_cur_tar'].rank(axis=0, method='min')
    dataset['rank_eucl_tar_prev'] = dataset.groupby('identifier')['eucl_tar_prev'].rank(axis=0, method='min')
    dataset['rank_eucl_tar_ves_fav'] = dataset.groupby('identifier')['eucl_ves_fav_tar'].rank(axis=0, method='min')
    dataset['rank_eucl_tar_mean'] = dataset.groupby('identifier')['eucl_ves_tar'].rank(axis=0, method='min')
    dataset['rank_eucl_port_fav_tar'] = dataset.groupby('identifier')['eucl_port_fav_tar'].rank(axis=0, method='min')
    
    #rename a column
    dataset.rename(columns={'n_visits_y': 'n_visits'}, inplace=True)
    
    #add some difference metrics
    for col in ['port_lat', 'port_long', 'n_visits', 'n_unique_vessels',
           'n_high_speed', 'n_medium_speed', 'n_Chemical/Oil Tanker', 'n_Container Ship', 'n_Crude Oil Tanker',
           'n_General Cargo Ship', 'n_Tanker', 'n_large_length', 'n_medium_length',
           'n_small_length', 'n_very large_length', 'n_large_depth',
           'n_medium_depth', 'n_small_depth', 'n_very large_depth', 'port_avg_distance_to_port', 'port_avg_travel_duration',
           'port_avg_stay_duration' ]:
        dataset[f'prev_curr_diff_{col}'] = abs(dataset[col] - dataset[f'previous_{col}'])
        dataset[f'curr_targ_diff_{col}'] = abs(dataset[f'target_{col}'] - dataset[col])
        dataset[f'fav_targ_diff_{col}'] = abs(dataset[f'target_{col}'] - dataset[f'vessel_fav_{col}'])   
    
    
    dataset['prev_speed_coor'] = (dataset['eucl_cur_prev']*111) / np.exp(dataset['previous_travel_duration'])
    dataset['prev_speed_sead'] = dataset['previous_seadistance'] / np.exp(dataset['previous_travel_duration'])
    dataset['prev2_speed_coor'] = (dataset['eucl_prev_prev2']*111) / np.exp(dataset['prev2_travel_duration'])
    dataset['prev2_speed_sead'] = dataset['prev2_prev_seadistance'] / np.exp(dataset['prev2_travel_duration'])
    
    dataset['exp_trav_prev_coor'] = np.log((dataset['eucl_cur_tar']*111) / dataset['prev_speed_coor'])
    dataset['exp_trav_prev_sead'] = np.log(dataset['seadistance'] / dataset['prev_speed_sead'])
    dataset['exp_trav_prev2_coor'] = np.log((dataset['eucl_cur_tar']*111) / dataset['prev2_speed_coor'])
    dataset['exp_trav_prev2_sead'] = np.log(dataset['seadistance'] / dataset['prev2_speed_sead'])
        
    return dataset

In [9]:
dataset = create_dataset(df)
print(dataset.shape)
dataset.head()

(4002500, 161)


,mmsi,ship_type,speed,length,depth,n_visits_x,n_unique_ports,vessel_avg_distance_to_port,vessel_avg_travel_duration,vessel_avg_stay_duration,...,vessel_fav_n_medium_depth,vessel_fav_n_small_depth,vessel_fav_n_very large_depth,vessel_fav_port_avg_distance_to_port,vessel_fav_port_avg_travel_duration,vessel_fav_port_avg_stay_duration,previous_seadistance,prev2_cur_seadistance,prev2_prev_seadistance,identifier
898236,538007476,None,None,None,None,22.0,8.0,6365.86,4.903427,3.593103,...,0.021538,0.004349,0.024498,7262.833463,1.846813,2.651755,271.74,7770.6,8053.45,538007476
898236,538007476,None,None,None,None,22.0,8.0,6365.86,4.903427,3.593103,...,0.021538,0.004349,0.024498,7262.833463,1.846813,2.651755,271.74,7770.6,8053.45,538007476
898236,538007476,None,None,None,None,22.0,8.0,6365.86,4.903427,3.593103,...,0.021538,0.004349,0.024498,7262.833463,1.846813,2.651755,271.74,7770.6,8053.45,538007476
898236,538007476,None,None,None,None,22.0,8.0,6365.86,4.903427,3.593103,...,0.021538,0.004349,0.024498,7262.833463,1.846813,2.651755,271.74,7770.6,8053.45,538007476
898236,538007476,None,None,None,None,22.0,8.0,6365.86,4.903427,3.593103,...,0.021538,0.004349,0.024498,7262.833463,1.846813,2.651755,271.74,7770.6,8053.45,538007476


In [10]:
dataset = add_features(dataset)
print(dataset.shape)
dataset.head()

(4002500, 321)


,mmsi,ship_type,speed,length,depth,n_visits_x,n_unique_ports,vessel_avg_distance_to_port,vessel_avg_travel_duration,vessel_avg_stay_duration,...,curr_targ_diff_port_avg_stay_duration,fav_targ_diff_port_avg_stay_duration,prev_speed_coor,prev_speed_sead,prev2_speed_coor,prev2_speed_sead,exp_trav_prev_coor,exp_trav_prev_sead,exp_trav_prev2_coor,exp_trav_prev2_sead
0,538007476,None,None,None,None,22.0,8.0,6365.86,4.903427,3.593103,...,0.334070,0.479751,10.491095,4.608236,23.333902,13.688079,7.231707,6.961958,6.432327,5.873278
1,538007476,None,None,None,None,22.0,8.0,6365.86,4.903427,3.593103,...,0.102403,0.248084,10.491095,4.608236,23.333902,13.688079,6.653999,7.429825,5.854618,6.341145
2,538007476,None,None,None,None,22.0,8.0,6365.86,4.903427,3.593103,...,0.227600,0.081919,10.491095,4.608236,23.333902,13.688079,3.446367,-inf,2.646987,-inf
3,538007476,None,None,None,None,22.0,8.0,6365.86,4.903427,3.593103,...,0.436934,0.582615,10.491095,4.608236,23.333902,13.688079,2.466596,-inf,1.667215,-inf
4,538007476,None,None,None,None,22.0,8.0,6365.86,4.903427,3.593103,...,0.042916,0.188598,10.491095,4.608236,23.333902,13.688079,3.141180,-inf,2.341799,-inf


### Mapping
- Use the same mapping as the one from port predictions

In [11]:
mappings = pickle.load(open('mappings.p', 'rb'))

for column in dataset.columns[dataset.dtypes == 'object']:
    if (column != 'identifier') and ('iso3' not in column) and ('portname' not in column):
        print(column)
        dataset[column] = dataset[column].map(mappings[column]).astype('category')

ship_type
speed
length
depth
prttype
prtsize
status
previous_prttype
previous_prtsize
previous_status
prev2_prttype
prev2_prtsize
prev2_status
vessel_fav_prttype
vessel_fav_prtsize
vessel_fav_status
target_prttype
target_prtsize
target_status


# Port prediction
- Load model and relevant columns
- Predict the next port
- Add the prediction/probability/rank to the dataset
- Only take the top 3 predictions (or top 1 depending on the DSS)
- The dataset is a lot smaller now since there are only 3 instances per mmsi instead of ~1800

In [12]:
port_model = pickle.load(open('port_model.p', 'rb'))
port_cols = pickle.load(open('port_cols.p', 'rb'))

In [13]:
dataset['prediction'] = port_model.predict(dataset[port_cols])
dataset = dataset.sort_values(['identifier', 'prediction'], ascending=[True, False])
dataset['rank'] = dataset.groupby('identifier').cumcount()+1
dataset['probability'] = dataset.groupby('identifier')['prediction'].apply(lambda x: softmax(x))

dataset = dataset[dataset['rank'] <= 3]

In [14]:
dataset[['mmsi', 'port_index', 'port_lat', 'port_long', 
        'target_port_index', 'target_port_lat', 'target_port_long', 'rank', 'probability', 'exit_datetime']]

,mmsi,port_index,port_lat,port_long,target_port_index,target_port_lat,target_port_long,rank,probability,exit_datetime
119430,201100124,342,44.4861,12.2867,2049,37.9470,22.94450,1,0.055618,2020-12-05 13:33:59
119353,201100124,342,44.4861,12.2867,1879,40.8947,29.19110,2,0.042569,2020-12-05 13:33:59
120024,201100124,342,44.4861,12.2867,3335,36.1308,-5.43892,3,0.033426,2020-12-05 13:33:59
2271659,205072000,797,51.3372,3.1979,3119,51.2135,4.38650,1,0.054904,2020-12-22 17:01:59
2271586,205072000,797,51.3372,3.1979,2945,51.1431,4.33410,2,0.047496,2020-12-22 17:01:59
...,...,...,...,...,...,...,...,...,...,...
1766652,755003340,2378,-34.8998,-56.1958,1595,-34.4526,-58.50530,2,0.010849,2020-12-22 23:01:51
1766530,755003340,2378,-34.8998,-56.1958,1367,-34.6026,-58.37470,3,0.010390,2020-12-22 23:01:51
3462492,770576272,2378,-34.8998,-56.1958,2378,-34.8998,-56.19580,1,0.819322,2020-12-22 18:22:05
3462111,770576272,2378,-34.8998,-56.1958,1595,-34.4526,-58.50530,2,0.020057,2020-12-22 18:22:05


# Travel Duration
- Load model and relevant columns
- Predict the travel duration
- Add it to the dataset

In [15]:
travel_model = pickle.load(open('travel_model.p', 'rb'))
travel_cols = pickle.load(open('travel_cols.p', 'rb'))

In [16]:
dataset['target_travel_duration'] = travel_model.predict(dataset[travel_cols])

# Stay Duration
- Load model and relevant columns
- Predict the stay duration
- Add it to the dataset

In [17]:
stay_model = pickle.load(open('stay_model.p', 'rb'))
stay_cols = pickle.load(open('stay_cols.p', 'rb'))

In [18]:
dataset['target_stay_duration'] = stay_model.predict(dataset[stay_cols])

# Extract relevant info
- Right now all (engineered) features are in the dataset
- We do not need all features for the DSS
- Select only the relevant information
- Prep ports: same as in preprocessing. This is used to add the full country name to each port (for DSS info)

In [19]:
info = dataset[['mmsi', 'port_index', 'port_lat', 'port_long', 'exit_datetime',
                   'target_port_index', 'target_port_lat', 'target_port_long', 
                   'target_travel_duration', 'target_stay_duration', 'probability', 'rank']]

In [20]:
def prep_ports():
    ports = pd.read_csv('ports.csv', delimiter=';', index_col='port_index', encoding='latin-1')
    print('old shape:', ports.shape)

    ports['iso3'] = ports['iso3'].fillna(ports['iso3_op'])
    ports[['prttype', 'prtsize', 'status', 'iso3']] = ports[['prttype', 'prtsize', 'status', 'iso3']].fillna('Unknown')
    ports = ports.drop(['code', 'maxdepth', 'maxlength', 'annualcapa', 
                        'iso3_op', 'latitude', 'longitude'], 1).reset_index()
    ports = ports[ports['port_index'].notnull()]

    ports = ports.replace('Sea Port', 'Sea')
    ports = ports.replace(['Restricted', 'Closed', 'Planned'], 'Unknown')
    ports = ports.replace('COG COD', 'COG')  #both congo
    ports = ports.replace('MOZ, MWI, ZWE', 'MOZ') #mozambique
    ports = ports.replace('MOZ, ZWE', 'MOZ')  #mozambique
    ports = ports.replace('ESH', 'MAR') #western sahara
    ports = ports.replace('JEY', 'FRA') #jersey
    ports = ports.replace('IMY', 'TUR') #some place near turkey
    ports.loc[ports['portname'] == 'Duqm', 'iso3'] = 'OMN' #the only port that we dont have iso3, but does occur in visits


    print('new shape:', ports.shape)
    
    return ports[['port_index', 'portname', 'country']]

In [21]:
ports = prep_ports()

old shape: (3576, 13)
new shape: (3575, 7)


In [23]:
info = pd.merge(info, ports, how='left', on='port_index')
info = pd.merge(info, ports.rename(columns={'portname': 'target_portname',
                                            'country': 'target_port_country',
                                            'port_index': 'target_port_index'}),
                  how='left', on='target_port_index')

In [24]:
info = info[['mmsi', 'country', 'port_index', 'portname', 'port_lat', 'port_long', 
             'exit_datetime', 'rank', 'probability', 'target_port_country', 
             'target_port_index', 'target_portname', 'target_port_lat', 
             'target_port_long', 'target_travel_duration', 'target_stay_duration']]

info['target_travel_duration'] = np.exp(info['target_travel_duration'])
info['target_stay_duration'] = np.exp(info['target_stay_duration'])

In [25]:
info.head(20)

,mmsi,country,port_index,portname,port_lat,port_long,exit_datetime,rank,probability,target_port_country,target_port_index,target_portname,target_port_lat,target_port_long,target_travel_duration,target_stay_duration
0,201100124,Italy,342,Porto Corsini,44.4861,12.2867,2020-12-05 13:33:59,1,0.055618,Greece,2049,Corinth,37.9470,22.94450,261.705011,17.030981
1,201100124,Italy,342,Porto Corsini,44.4861,12.2867,2020-12-05 13:33:59,2,0.042569,Turkey,1879,Kartal,40.8947,29.19110,316.185551,38.150373
2,201100124,Italy,342,Porto Corsini,44.4861,12.2867,2020-12-05 13:33:59,3,0.033426,NaN,3335,Algeciras,36.1308,-5.43892,302.669652,13.249674
3,205072000,Belgium,797,Zeebrugge,51.3372,3.1979,2020-12-22 17:01:59,1,0.054904,Belgium,3119,Antwerpen (Antwerp),51.2135,4.38650,26.060040,46.907591
4,205072000,Belgium,797,Zeebrugge,51.3372,3.1979,2020-12-22 17:01:59,2,0.047496,Belgium,2945,Hemiksem (Hemixen),51.1431,4.33410,29.587440,28.357654
5,205072000,Belgium,797,Zeebrugge,51.3372,3.1979,2020-12-22 17:01:59,3,0.017580,Belgium,2513,Ghent,51.0504,3.73930,28.716933,18.439092
6,205199000,Belgium,2519,Doornzelle,51.1212,3.7575,2020-12-17 02:02:03,1,0.075134,Belgium,2945,Hemiksem (Hemixen),51.1431,4.33410,48.758311,32.664276
7,205199000,Belgium,2519,Doornzelle,51.1212,3.7575,2020-12-17 02:02:03,2,0.042615,Belgium,101,Antwerp,51.2333,4.46667,37.248705,41.871003
8,205199000,Belgium,2519,Doornzelle,51.1212,3.7575,2020-12-17 02:02:03,3,0.037950,Belgium,3119,Antwerpen (Antwerp),51.2135,4.38650,38.551584,45.418593
9,205303000,Belgium,797,Zeebrugge,51.3372,3.1979,2020-12-14 00:00:02,1,0.112294,France,2701,Boulogne Sur Mer,50.7305,1.60010,38.263221,18.052415


In [26]:
info['target_entry_datetime'] = (info['exit_datetime'] + 
                                 pd.to_timedelta(info['target_travel_duration'], 'hours')).dt.round('1min')
info['target_exit_datetime'] = (info['target_entry_datetime'] + 
                                 pd.to_timedelta(info['target_stay_duration'], 'hours')).dt.round('1min')

In [27]:
info.reset_index(drop=True, inplace=True)

In [28]:
info = info[info['rank']==1].drop('rank', 1)

In [29]:
info.head()

,mmsi,country,port_index,portname,port_lat,port_long,exit_datetime,probability,target_port_country,target_port_index,target_portname,target_port_lat,target_port_long,target_travel_duration,target_stay_duration,target_entry_datetime,target_exit_datetime
0,201100124,Italy,342,Porto Corsini,44.4861,12.2867,2020-12-05 13:33:59,0.055618,Greece,2049,Corinth,37.9470,22.9445,261.705011,17.030981,2020-12-16 11:16:00,2020-12-17 04:18:00
3,205072000,Belgium,797,Zeebrugge,51.3372,3.1979,2020-12-22 17:01:59,0.054904,Belgium,3119,Antwerpen (Antwerp),51.2135,4.3865,26.060040,46.907591,2020-12-23 19:06:00,2020-12-25 18:00:00
6,205199000,Belgium,2519,Doornzelle,51.1212,3.7575,2020-12-17 02:02:03,0.075134,Belgium,2945,Hemiksem (Hemixen),51.1431,4.3341,48.758311,32.664276,2020-12-19 02:48:00,2020-12-20 11:28:00
9,205303000,Belgium,797,Zeebrugge,51.3372,3.1979,2020-12-14 00:00:02,0.112294,France,2701,Boulogne Sur Mer,50.7305,1.6001,38.263221,18.052415,2020-12-15 14:16:00,2020-12-16 08:19:00
12,205394390,Germany,1556,Duisburg,51.4073,6.7527,2020-12-22 11:02:04,0.235217,Belgium,797,Zeebrugge,51.3372,3.1979,27.128850,32.381580,2020-12-23 14:10:00,2020-12-24 22:33:00


In [30]:
info['unique_port_identifier'] = info['portname'] + ' (' + info['port_index'].astype(str) + ')'
info['unique_target_port_identifier'] = info['target_portname'] + ' (' + info['target_port_index'].astype(str) + ')'

### Since the dataset is massive for only a few predictions it was needed to run this code a few times. So this part retrieves the old predictions and adds the new ones and saves them

In [31]:
combined_info = pd.read_csv('DSS_input.csv')
combined_info = pd.concat([combined_info, info])
combined_info = combined_info.drop_duplicates(subset='mmsi')
combined_info.shape

(4755, 19)

In [32]:
combined_info['exit_datetime'] = pd.to_datetime(combined_info['exit_datetime'])
combinded_info = combined_info.sort_values('exit_datetime').reset_index(drop=True)

In [33]:
combined_info.to_csv('DSS_input.csv', index=False)